In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("trabajo_final").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
!sh directorio.sh

El directorio ya existe warehouse/raw


In [3]:
!sh dir_hdfs.sh

El directorio /user/warehouse ya existe.
El directorio /user/warehouse/bronze ya existe.
El directorio /user/warehouse/silver ya existe.
El directorio /user/warehouse/gold ya existe.
Se crearon los directorios correctamente


In [ ]:
!pip3 install shapely
!pip3 install geopandas


In [23]:
# Crear el dataset crudo

!python3 01_gen_bd/create_data.py

In [27]:
!hdfs dfs -copyFromLocal warehouse/raw/40000303042024.parquet /user/warehouse/bronze/data_prueba_3.parquet

copyFromLocal: `/user/warehouse/data_prueba_3.parquet': File exists


In [26]:
from pyspark import SparkFiles

file: str = "hdfs:///user/warehouse/data_prueba_3.parquet"
df = spark.read.parquet(file)
df.describe()

DataFrame[summary: string, partition_date: string, order_id: string, commune: string, customer_id: string, employee_id: string, event_date: string, event_day: string, event_hour: string, event_minute: string, event_month: string, event_second: string, event_year: string, latitude: string, longitude: string, neighborhood: string, quantity_products: string, __index_level_0__: string]

In [39]:
df.write.parquet("hdfs:///tmp/dia_1.parquet")

In [40]:
!hdfs dfs -ls /tmp/

Found 1 items
drwxr-xr-x   - root supergroup          0 2024-06-20 03:29 /tmp/dia_1.parquet


In [41]:
!hdfs dfs -ls /tmp/dia_1.parquet


Found 2 items
-rw-r--r--   1 root supergroup          0 2024-06-20 03:29 /tmp/dia_1.parquet/_SUCCESS
-rw-r--r--   1 root supergroup      74918 2024-06-20 03:29 /tmp/dia_1.parquet/part-00000-05f0d5c0-8358-40ff-8d14-3ec478c8b912-c000.snappy.parquet


In [52]:
!hdfs dfs -ls /tmp/

In [53]:
spark.sql("CREATE DATABASE IF NOT EXISTS base_comunas;")

DataFrame[]

In [54]:
modo: str = "append"
df.write.mode(modo).saveAsTable("base_comunas.dias")

In [55]:
spark.sql("DESCRIBE base_dias;")

DataFrame[col_name: string, data_type: string, comment: string]